In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
import os
import re
import time
import torch

from torch import nn, optim
from torch.utils import data
from torch.utils.data import TensorDataset, DataLoader
from glob import iglob, glob
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
"""tensors = []
for path in paths:
    tensor = torch.load(path).numpy()
    tensors.append(tensor)
tensors = np.array(tensors).astype('float32')
tensors.shape

tensors = tensors.reshape((len(tensors), -1))
print(tensors.shape)

batch_size = 16
seq_len = 128

full_seq = tensors.shape[0] // (seq_len * batch_size) * (seq_len * batch_size)
print(full_seq)

tensors = tensors[:full_seq].reshape((batch_size, -1, tensors.shape[1]))
print(tensors.shape)"""

"tensors = []\nfor path in paths:\n    tensor = torch.load(path).numpy()\n    tensors.append(tensor)\ntensors = np.array(tensors).astype('float32')\ntensors.shape\n\ntensors = tensors.reshape((len(tensors), -1))\nprint(tensors.shape)\n\nbatch_size = 16\nseq_len = 128\n\nfull_seq = tensors.shape[0] // (seq_len * batch_size) * (seq_len * batch_size)\nprint(full_seq)\n\ntensors = tensors[:full_seq].reshape((batch_size, -1, tensors.shape[1]))\nprint(tensors.shape)"

In [3]:
class S2FDatasetRAM(data.Dataset):
    '''
    Sequence of Frames to one frame dataset.
    Load all data into RAM at once.
    '''

    def __init__(self, datadir, seq_len):
        '''
        Initialization
        Args:
            datadir: directory of serialized tensors
            seq_len: timestep length of tensors
            batch_size: divide a sequence to n_batch sequences
        '''
        self.paths = glob(datadir + '/*.pkl')
        path_num = len(self.paths)
        full_seq = path_num // seq_len * seq_len
        # only want full size batches
        self.paths = self.paths[: full_seq]

        self.seq_len = seq_len
        # total length of all tensors
        self.length = len(self.paths)

        # load all tensor into RAM
        tensors = []
        for path in self.paths:
            tensor = torch.load(path).numpy()
            tensors.append(tensor)
        tensors = np.array(tensors).astype('float32')
        self.tensors = tensors.reshape((self.length, -1))

    def __len__(self):
        '''
        Denotes the total number of samples
        '''
        return self.length - self.seq_len

    def __getitem__(self, index):
        '''
        Generates one sample of data
        '''
        X = self.tensors[index: index+self.seq_len]
        y = self.tensors[index+self.seq_len]

        return X, y

In [4]:
datadir = 'tensor_dataset/full_year_2018_15min/tensors'
paths = glob(datadir + '/*.pkl')

In [5]:
dataset = S2FDatasetRAM(datadir, 96)

In [6]:
loader = data.DataLoader(dataset, shuffle=False , batch_size = 20)
d_iter = iter(loader)
a = d_iter.next()
print(a[0].shape, a[1].shape)

torch.Size([20, 96, 14283]) torch.Size([20, 14283])
